## Note: You may need to adjust some paths in the following code. I moved some stuff around last minute!

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os

from overlapped_classes import overlapped_classes
import json
from PIL import Image
import scipy

from torchattacks import FGSM, PGD


In [2]:
with open("../mappings/objectnet_to_imagenet_1k.json") as f:
    data = json.load(f)

with open("../mappings/pytorch_to_imagenet_2012_id.json") as f:
    idxMap = json.load(f)
        
with open("../mappings/folder_to_objectnet_label.json") as f:
    folderMap = json.load(f)

with open('imagenet_classes.txt') as f:
# with open('../mappings/imagenet_to_label_2012_v2.txt') as f:    
    labels = [line.strip() for line in f.readlines()]    

In [3]:
# inverting the folderMap
dirMap = {}
for u, v in folderMap.items():
    dirMap[v]= u

In [4]:
from torchvision import models
dir(models)

# models.googlenet(pretrained=True)] #,

models_ = [models.vgg19(pretrained=True), 
           models.resnet152(pretrained=True), models.inception_v3(pretrained=True)]
#            models.mnasnet1_0(pretrained=True)] #, models.resnext101_32x8d(pretrained=True), models.densenet161(pretrained=True)]



In [15]:

# model = models.alexnet(pretrained=True)
# model = models.resnet152(pretrained=True)
# model = models.inception_v3(pretrained=True)
# model = models.googlenet(pretrained=True)

# for model_name in models_:
#     model = eval(model_name)
#     model.eval()

In [5]:
def predict_image(image_path, box, attack, label, draw=False, kind='Box'):
    # Define transformations for the image, should (note that imagenet models are trained with image size 224)
    transform = transforms.Compose([
#         transforms.CenterCrop(224),
        transforms.ToTensor(),
#         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))    
    ])
    
    
    if kind == 'Crop':
        transform = transforms.Compose([
            transforms.CenterCrop(224),
            transforms.ToTensor(),
    #         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))    
        ])
    
    
    img = Image.open(image_path).convert("RGB")

    if kind == 'Box':
        xl, yl, xr, yr = box
        img = img.crop((xl, yl, xr, yr))   #((left, top, right, bottom)) 
        img = img.resize((224, 224))
        

    if kind == 'Full':
        img = img.resize((224, 224))
        

    img_t = transform(img).float()

    if draw:
        plt.imshow(img) #img_t.permute((2,1,0)) )
        plt.show()

    image_tensor = img_t.unsqueeze_(0)

    if torch.cuda.is_available():
        image_tensor.cuda()

    # perform the attack        
#     import pdb; pdb.set_trace()
#     inputs_adv = attack(image_tensor, label)#.cuda()    
    imin = image_tensor.min() 
    imax = image_tensor.max()     
    image_tensor = (image_tensor - imin) / (imax - imin)
    inputs_adv = attack(image_tensor.detach(), torch.tensor([label]))
    inputs_adv = inputs_adv * (imax  - imin) + imin
    output = model(inputs_adv)
    
    # Predict the class of the image
#     output = model(image_tensor)
    
    confs, indices = torch.sort(output.data, descending=True)
    confs[0] = confs[0]/sum(confs[0])
    return np.abs(confs[0][:5]), indices[0][:5]  # index

In [8]:
eps = 8/255
# perform the attack
# if attack_type.upper() == 'FGSM':
# else:    
#     attack = PGD(net, eps=eps, alpha=8/255, iters=5)    


In [9]:
kind = 'Box'
for model in models_:
    model_name = model._get_name() #eval(model_name)
    model.eval()

#     attack = FGSM(model, eps=eps)    
    attack = PGD(model, eps=eps, alpha=8/255, iters=5)    
    
    print(model_name + '\n')
    
    res_top1 = []
    res_top5 = []

    save_file = './outputs_attack/' + model_name + '.txt'

    for n, category in enumerate(data):

        txtfile = os.path.join('../Annotations', dirMap[category] + '.txt')
        if not os.path.exists(txtfile):
            continue

        with open(txtfile) as f:
            boxes = [line.strip() for line in f.readlines()]    

        # writing to the file
        with open(save_file, 'a') as ff:
            ff.write(f"{category} ---------------------------------------------------------------- \n")


        count = [0, 0]
        lines = 0
        for im in boxes[0:10]:
            ss = im.split(' ')
            fName = ss[0]
            if len(ss) > 1:
                lines += 1
                coords = (int(i) for i in ss[1:] if i)
                
                # find the true label
                trueIdx = -10
                trueIdx = labels.index(data[category].split(';')[0])
#                 print(trueIdx)
                
#                 import pdb; pdb.set_trace()
                confs, idx = predict_image(os.path.join('../images/' + dirMap[category] + '/', fName), coords, attack, trueIdx , False, kind)

                # top 1
                
                count[0] += 1 if labels[idx[0]] in data[category] else 0  

                # top 5
                flag = False
                for i in idx[0:]:
                      flag = flag or (labels[i] in data[category])
                count[1] += 1 if flag else 0

                # writing to the file
                with open(save_file, 'a') as ff:
                    ff.write(f"{fName} {int(idx[0])} {int(idx[1])} {int(idx[2])} {int(idx[3])} {int(idx[4])}           {confs[0]:.2f} {confs[1]:.2f} {confs[2]:.2f} {confs[3]:.2f} {confs[4]:.2f} \n")

        accs = np.array(count)*100/lines

        print(f"{n} -> {category}: top 1: {accs[0]:.2f}  -  top 5: {accs[1]:.2f}           [num imgs: {lines}]")  

        with open(save_file, 'a') as ff:
            ff.write(f"{n} -> {category}: top 1: {accs[0]:.2f}  -  top 5: {accs[1]:.2f}           [num imgs: {lines}] \n")  

        res_top1.append(accs[0])        
        res_top5.append(accs[1])

    print(sum(res_top1)/len(res_top1))
    print(sum(res_top5)/len(res_top5))

VGG

0 -> Alarm clock: top 1: 0.00  -  top 5: 10.00           [num imgs: 10]
1 -> Backpack: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
2 -> Banana: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
3 -> Band Aid: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
4 -> Basket: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
5 -> Bath towel: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
6 -> Beer bottle: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
7 -> Bench: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
8 -> Bicycle: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
9 -> Binder (closed): top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
10 -> Bottle cap: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
11 -> Bread loaf: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
12 -> Broom: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
13 -> Bucket: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
14 -> Butcher's knife: top 1: 0.

KeyboardInterrupt: 

In [ ]:
txtfile

print(sum(res_top1)/len(res_top1))
print(sum(res_top5)/len(res_top5))

In [157]:
# eval(model)
# int(idx[0])
print(f"{1.911333:.2f}")
# print(f'{val:.2f}')

confs, idx = predict_image(os.path.join('../images/' + dirMap[category] + '/', fName), coords, False)
# confs / sum(confs)

1.91


ValueError: not enough values to unpack (expected 4, got 0)

In [81]:
eval(model_name)

'models.vgg19(pretrained=True)'

In [82]:
models_ = [models.googlenet(pretrained=True), models.alexnet(pretrained=True), models.vgg19(pretrained=True), 
           models.resnet152(pretrained=True), models.inception_v3(pretrained=True), 
           models.mnasnet(pretrained=True), models.resnext101_32x8d(pretrained=True)]

TypeError: 'module' object is not callable

In [87]:
m = models.mnasnet1_0(pretrained=True)

Downloading: "https://download.pytorch.org/models/mnasnet1.0_top1_73.512-f206786ef8.pth" to /Users/ali/.cache/torch/checkpoints/mnasnet1.0_top1_73.512-f206786ef8.pth
100.0%


In [101]:
# dir(models)
models_[0]._get_name()

'GoogLeNet'

In [26]:
kind = 'Box'
for model in models_:
    model_name = model._get_name() #eval(model_name)
    model.eval()

    attack = FGSM(model, eps=eps)    
    
    
    print(model_name + '\n')
    
    res_top1 = []
    res_top5 = []

    save_file = './outputs_attack_tmp/' + model_name + '.txt'

    for n, category in enumerate(data):

        txtfile = os.path.join('../Annotations', dirMap[category] + '.txt')
        if not os.path.exists(txtfile):
            continue

        with open(txtfile) as f:
            boxes = [line.strip() for line in f.readlines()]    

        # writing to the file
        with open(save_file, 'a') as ff:
            ff.write(f"{category} ---------------------------------------------------------------- \n")


        count = [0, 0]
        lines = 0
        for im in boxes[0:10]:
            ss = im.split(' ')
            fName = ss[0]
            if len(ss) > 1:
                lines += 1
                coords = (int(i) for i in ss[1:] if i)
                
                # find the true label
                trueIdx = -10
                trueIdx = labels.index(data[category].split(';')[0])
#                 print(trueIdx)
                
#                 import pdb; pdb.set_trace()
                confs, idx = predict_image(os.path.join('../images/' + dirMap[category] + '/', fName), coords, attack, trueIdx , False, kind)

                # top 1
                
                count[0] += 1 if labels[idx[0]] in data[category] else 0  

                # top 5
                flag = False
                for i in idx[0:]:
                      flag = flag or (labels[i] in data[category])
                count[1] += 1 if flag else 0

                # writing to the file
                with open(save_file, 'a') as ff:
                    ff.write(f"{fName} {int(idx[0])} {int(idx[1])} {int(idx[2])} {int(idx[3])} {int(idx[4])}           {confs[0]:.2f} {confs[1]:.2f} {confs[2]:.2f} {confs[3]:.2f} {confs[4]:.2f} \n")

        accs = np.array(count)*100/lines

        print(f"{n} -> {category}: top 1: {accs[0]:.2f}  -  top 5: {accs[1]:.2f}           [num imgs: {lines}]")  

        with open(save_file, 'a') as ff:
            ff.write(f"{n} -> {category}: top 1: {accs[0]:.2f}  -  top 5: {accs[1]:.2f}           [num imgs: {lines}] \n")  

        res_top1.append(accs[0])        
        res_top5.append(accs[1])

    print(sum(res_top1)/len(res_top1))
    print(sum(res_top5)/len(res_top5))

VGG

0 -> Alarm clock: top 1: 0.00  -  top 5: 10.00           [num imgs: 10]
1 -> Backpack: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
2 -> Banana: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
3 -> Band Aid: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
4 -> Basket: top 1: 0.00  -  top 5: 10.00           [num imgs: 10]
5 -> Bath towel: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
6 -> Beer bottle: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
7 -> Bench: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
8 -> Bicycle: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
9 -> Binder (closed): top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
10 -> Bottle cap: top 1: 0.00  -  top 5: 10.00           [num imgs: 10]
11 -> Bread loaf: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
12 -> Broom: top 1: 0.00  -  top 5: 10.00           [num imgs: 10]
13 -> Bucket: top 1: 0.00  -  top 5: 0.00           [num imgs: 10]
14 -> Butcher's knife: top 1: